In [31]:
import cv2
import numpy as np
import os

from scipy.cluster.vq import kmeans, vq
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

In [58]:
#!nvidia-smi
#!pip install cupy-cuda112
#!pip install cupy
#chainer.print_runtime_info()

In [59]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def imglist(path):
    return [os.path.join(path, f) for f in os.listdir(path)]

In [3]:
def getAllRectangles(im_x,im_y,step,window):
    
    #Initialise Patch Size 
    minx = 0
    maxx = im_x
    miny = 0
    maxy =im_y
    step_x = step
    step_y = step
    window_width = window
    window_height = window
    rectangle = [] 
    x = minx;
    y = miny;
    hasNext = True

    while hasNext:
        nextX = x + step_x;
        nextY = y;
        if (nextX + window_width > maxx):
            nextX = minx;
            nextY += step_y;
        rec_dim = [x, y, window_width, window_height]
        #print(rec_patch)
        rectangle.append(rec_dim);
        x = nextX;
        y = nextY;

        if (y + window_height > maxy):
            hasNext = False
    #print("All rectangular patches retrieved.......")
    return rectangle

In [4]:
def featureExtraction(im, rectangle):
    feature = []
    for x,y,w,h in rectangle:
        patch_img = im[x:x+w,y:y+w]
        img_array = np.array(patch_img)
        flat_arr = img_array.ravel()
        vector = flat_arr.tolist()
        feature.append(vector)
    return feature

In [5]:
def descriptorFormation(descriptor_list):
    descriptors = descriptor_list[0][1]
    img_count=0
    #Stacking
    for image_path, descriptor in descriptor_list[1:]:
        img_count+=1
        #Array of all features
        descriptors = np.vstack((descriptors, descriptor))
        #print("Stacking of Descriptors of image {} complete......".format(img_count))
    print("Descriptors stacked successfully!")
    descriptors_float = descriptors.astype(float)
    return(descriptors_float)

In [6]:
def getDescriptors(image_paths):
    descriptor_list = []
    img_count=0
    
    for image_path in image_paths:
        img_count+=1
        im = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        m, n = im.shape
        rectangle = getAllRectangles(m,n,4,8) 
        feature = featureExtraction(im, rectangle)
        #print("Image features extracted for image {}......".format(img_count))
        descriptor_list.append((image_path, feature))
    print("Feature extraction done!")
    return(descriptor_list)

In [15]:
def quantisation(image_paths, descriptor_list, k, codebook, scaler):
    im_features = np.zeros((len(image_paths), k), "float32")
    for i in range(len(image_paths)):
        descriptors = scaler.transform(descriptor_list[i][1])
        codes, distances = vq(descriptors, codebook)
        for code in codes:
            im_features[i][code] += 1
    return(im_features)       

In [8]:
train_path = 'images/training'
training_names = os.listdir(train_path)
image_paths = []
#Class label for all images
image_classes = []
class_id = 0

for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imglist(dir)
    image_paths+=class_path
    image_classes+=[class_id]*len(class_path)
    class_id+=1
     
descriptor_list = getDescriptors(image_paths)

Feature extraction done!


In [9]:
descriptors_float = descriptorFormation(descriptor_list)

Descriptors stacked successfully!


In [10]:
print(len(descriptors_float))

6016463


In [11]:
#Normalising patches
scaler = StandardScaler()
normalised_descriptors = scaler.fit_transform(descriptors_float)

In [12]:
#Sample descriptors
sampled_descriptors = normalised_descriptors[np.random.choice(normalised_descriptors.shape[0], 500000, replace=False), :]

In [13]:
k = 500
voc, variance = kmeans(sampled_descriptors, k, 1)

In [16]:
# Calculate the histogram of features and represent them as vector
im_features = quantisation(image_paths,descriptor_list, k, voc, scaler)

In [28]:
#print(im_features[-1:])

### One-vs-Rest Classifier

In [32]:
min_max_scaler = preprocessing.MinMaxScaler()
im_features_minmax = min_max_scaler.fit_transform(im_features)

model = LogisticRegression(multi_class='ovr', max_iter=100)
model.fit(im_features_minmax, image_classes)

In [34]:
score = model.score(im_features_minmax, image_classes)
print(score)

0.8933333333333333


### To Validate using Test data

In [ ]:
test_path = 'images/testing'
testing_names = os.listdir(test_path)

test_image_paths = []
test_image_classes = []
test_class_id = 0

for testing_name in testing_names:
    test_dir = os.path.join(test_path, testing_name)
    test_image_paths.append(test_dir)
    
test_descriptor_list = getDescriptors(test_image_paths)
test_descriptors_float = descriptorFormation(test_descriptor_list)

In [ ]:
test_im_features = quantisation(image_paths,descriptor_list)
test_im_features = stdSlr.transform(test_im_features)